In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import r2_score
import lightgbm as lgb


train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_new = pd.read_csv('./data/train_new.csv')
test_new = pd.read_csv('./data/test_new.csv')

train = pd.concat([train, train_new], axis=1)
test = pd.concat([test, test_new], axis=1)
train = train.fillna(train.mean())
test = test.fillna(test.mean())

features = [c for c in train.columns if c not in ["Y1", "Y2"]]

# X = train[features].values
# X_test = test[features].values
y2 = train["Y2"].values

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X = scaler.fit_transform(train[features].values)
X_test = scaler.transform(test[features].values)

n_features = X.shape[1]

mlp_model = Sequential([
    Input(shape=(n_features,)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])


mlp_model.compile(optimizer='adam', loss='mse')

X_train, X_val, y2_train, y2_val = train_test_split(X, y2, test_size=0.2, random_state=42, shuffle = True)
# split = int(0.8 * len(X))
# X_train, X_val = X[:split], X[split:]
# y2_train, y2_val = y2[:split], y2[split:]
#50 and 64
hist = mlp_model.fit(X_train, y2_train, 
                     epochs=200, 
                     batch_size=128, 
                     validation_data=(X_val, y2_val), 
                     verbose=1)

y2_pred = mlp_model.predict(X_val)
r2 = r2_score(y2_val, y2_pred)
print(f"Validation R² Score: {r2}")

2025-09-21 11:12:09.655959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
483/500 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4226

KeyboardInterrupt: 